In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [3]:

# Get the Neo4j driver
from storage import Storage
s = Storage()

from ha_utils import HeaderAnalysis
ha = HeaderAnalysis(verbose=False)

from scrape_utils import WebScrapingUtilities
wsu = WebScrapingUtilities(s=s)
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']

from cypher_utils import CypherUtilities
cu = CypherUtilities(uri=uri, user=user, password=password, driver=None, s=s, ha=ha)

In [4]:

from neo4j.exceptions import ServiceUnavailable

try:
    version_str = cu.driver.verify_connectivity()
    print(f'======== {version_str} ========')
    
    from hc_utils import HeaderCategories
    hc = HeaderCategories(cu=cu, verbose=False)
    
    from section_utils import SectionUtilities
    su = SectionUtilities(s=s, ha=ha, cu=cu, verbose=False)
    
    from lr_utils import LrUtilities
    lru = LrUtilities(ha=ha, cu=cu, hc=hc, verbose=False)
    
    from crf_utils import CrfUtilities
    crf = CrfUtilities(ha=ha, hc=hc, cu=cu, verbose=False)
    
    import warnings
    warnings.filterwarnings('ignore')
except ServiceUnavailable as e:
    # print(str(e).strip())
    raise ServiceUnavailable('You need to start Neo4j as a console')
except Exception as e:
    print(e.__class__)

======== Neo4j/4.4.7 ========


In [5]:

import time
import humanize
from pandas import DataFrame
import os
from datetime import datetime
import winsound

duration = 1000  # milliseconds
freq = 440  # Hz
print(f'Last run on {datetime.now()}')

Last run on 2022-12-15 18:53:05.423956



---
# Training

In [320]:

# Is-header must be retrained before parts-of-speech
t0 = time.time()
lru.build_isheader_logistic_regression_elements(verbose=False)
lru.retrain_isheader_classifier(verbose=False)
duration_str = humanize.precisedelta(time.time() - t0, minimum_unit='seconds', format='%0.0f')
winsound.Beep(freq, duration)
print(f'Is-header classifier retrained in {duration_str}')

Is-header classifier retrained in 16 seconds


In [321]:

t0 = time.time()
lru.build_pos_logistic_regression_elements(verbose=False)
crf.retrain_pos_classifier(verbose=False)
duration_str = humanize.precisedelta(time.time() - t0, minimum_unit='seconds', format='%0.0f')
winsound.Beep(freq, duration)
print(f'Parts-of-speech classifier retrained in {duration_str}')

Parts-of-speech classifier retrained in 8 minutes and 55 seconds


In [322]:

# Rebuild the classifer from the quals dictionary
t0 = time.time()
lru.build_isqualified_logistic_regression_elements(verbose=False)
lru.retrain_isqualified_classifier(verbose=True)
duration_str = humanize.precisedelta(time.time() - t0, minimum_unit='seconds', format='%0.0f')
winsound.Beep(freq, duration)
print(f'Is-qualified classifer retrained in {duration_str}')

I have 6,568 hand-labeled qualification strings in here
Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\basic_quals_dict.pkl
Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\basic_quals_df.pkl
Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\ISQUALIFIED_VOCAB.pkl
Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\ISQUALIFIED_TT.pkl
Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\ISQUALIFIED_LR.pkl
Retraining complete
Is-qualified classifer retrained in 21 seconds


In [324]:

navigable_parent_cypher_str = '''
    MATCH (np:NavigableParents {{navigable_parent: '{}'}})
    ''' + cu.return_everything_str + ';'
cypher_str = f'''
    MATCH (fn:FileNames)
    WHERE
        fn.percent_fit < 0.4 AND
        ((fn.is_closed IS NULL) OR (fn.is_closed = false)) AND
        ((fn.is_verified IS NULL) OR (fn.is_verified = false)) AND
        ((fn.is_opportunity_application_emailed IS NULL) OR
        (fn.is_opportunity_application_emailed = false))
    RETURN
        fn.percent_fit AS percent_fit,
        fn.file_name AS file_name,
        fn.posting_url AS url
    ORDER BY fn.percent_fit ASC;'''
row_objs_list = []
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
files_list = []
if row_objs_list:
    files_list = DataFrame(row_objs_list).file_name.tolist()
print(f'Only {len(files_list)} more mis-estimated minimum-requirements-met percentages to go!')

Only 0 more mis-estimated minimum-requirements-met percentages to go!



----

In [293]:

# file_name = 'eca2876d9f354127_Security_Software_Engineer_L5_Detection_Engineering_Remote_Indeed_com.html'
file_name = files_list.pop()
file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
page_soup = wsu.get_page_soup(file_path)
div_soup = page_soup.find_all(name='div', id='jobDescriptionText')[0]
child_strs_list = ha.get_navigable_children(div_soup, [])
cu.ensure_filename(file_name, verbose=False)
cu.populate_from_child_strings(child_strs_list, file_name, verbose=False)
print(file_name)

HPC_User_Environment_Architect_Remote_Friendly_San_Francisco_Bay_Area_CA_Indeed_com.html


In [294]:

child_tags_list = ha.get_child_tags_list(child_strs_list)
is_header_list = []
for is_header, child_str in zip(ha.get_is_header_list(child_strs_list), child_strs_list):
    if is_header is None:
        probs_list = lru.ISHEADER_PREDICT_PERCENT_FIT(child_str)
        idx = probs_list.index(max(probs_list))
        is_header = [True, False][idx]
    is_header_list.append(is_header)
feature_dict_list = hc.get_feature_dict_list(child_tags_list, is_header_list, child_strs_list)
feature_tuple_list = []
for feature_dict in feature_dict_list:
    feature_tuple_list.append(hc.get_feature_tuple(feature_dict, lru.pos_lr_predict_single))
crf_list = crf.CRF.predict_single(crf.sent2features(feature_tuple_list))
db_pos_list = []
for navigable_parent in child_strs_list:
    db_pos_list = cu.append_parts_of_speech_list(navigable_parent, pos_list=db_pos_list)
pos_list, indices_list = su.visualize_basic_quals_section(crf_list, child_strs_list, db_pos_list=db_pos_list, verbose=True)

['O-TS', 'O-TS', 'H-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'H-TS', 'O-TS', 'O-CS', 'O-TS', 'H-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'H-PQ', 'O-PQ', 'O-PQ', 'O-PQ', 'O-PQ', 'O-PQ', 'O-PQ', 'O-PQ', 'O-PQ', 'O-PQ', 'O-PQ', 'O-IP', 'O-CS', 'O-CS', 'O-SP', 'O-IP', 'O-TS', 'O-OL', 'O-TS', 'O-LN', 'O-TS', 'O-LN']
[19, 20, 21, 22, 23, 24, 25, 26]


[19, 20, 21, 22, 23, 24, 25, 26]


In [ ]:
raise

In [317]:

# Display the context of an individual child string
idx = 38
print(indices_list); child_str = child_strs_list[idx]; pos_symbol = pos_list[idx]; basic_quals_dict = s.load_object('basic_quals_dict')
print(str(basic_quals_dict[child_str]) + '\n' if(child_str in basic_quals_dict) else '', end=''); print(f'{idx} {pos_symbol}) {child_str}')

[19, 20, 21, 22, 23, 24, 25, 26]
38 O-IP) <b>Want to learn more about Berkeley Lab's Culture, Benefits and answers to FAQs? Please visit:</b>


In [279]:

# Hand-label this particular child string in the quals dictionary
basic_quals_dict = s.load_object('basic_quals_dict')
basic_quals_dict[child_str] = 0
s.store_objects(basic_quals_dict=basic_quals_dict); print(f'"{child_str}" in basic_quals_dict: {basic_quals_dict[child_str]}')

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\basic_quals_dict.pkl
"<li>You have experience working in a continuous delivery, cloud-based environment (Jenkins and/or Spinnaker desired, but not required).</li>" in basic_quals_dict: 0


In [296]:

def do_cypher_tx(tx, navigable_parent, verbose=False):
    cypher_str = """MATCH (np:NavigableParents {navigable_parent: $navigable_parent})
        SET
            np.is_header = 'False',
            np.is_task_scope = 'True',
            np.is_minimum_qualification = 'False',
            np.is_preferred_qualification = 'False',
            np.is_educational_requirement = 'False',
            np.is_legal_notification = 'False',
            np.is_other = 'False',
            np.is_corporate_scope = 'False',
            np.is_job_title = 'False',
            np.is_office_location = 'False',
            np.is_job_duration = 'False',
            np.is_supplemental_pay = 'False',
            np.is_interview_procedure = 'False',
            np.is_posting_date = 'False'
        """ + cu.return_everything_str + ';'
    results_list = tx.run(query=cypher_str, parameters={'navigable_parent': navigable_parent})

    return [dict(record.items()) for record in results_list]
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(do_cypher_tx, navigable_parent=child_str)
row_objs_list

[{'navigable_parent': '<li>Mentor early career staff members</li>', 'is_header': 'False', 'is_task_scope': 'True', 'is_qualification': None, 'is_minimum_qualification': 'False', 'is_preferred_qualification': 'False', 'is_legal_notification': 'False', 'is_job_title': 'False', 'is_office_location': 'False', 'is_job_duration': 'False', 'is_supplemental_pay': 'False', 'is_educational_requirement': 'False', 'is_interview_procedure': 'False', 'is_corporate_scope': 'False', 'is_posting_date': 'False', 'is_other': 'False'}]

In [ ]:

# Show what's in the database already for this html string
# print(navigable_parent_cypher_str.format(child_str))
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, navigable_parent_cypher_str.format(child_str))
row_objs_list

In [42]:

# Remove this particular child string from the quals dictionary
basic_quals_dict = s.load_object('basic_quals_dict')
child_str = child_strs_list[idx]
basic_quals_dict.pop(child_str)
# basic_quals_dict[child_str] = 0
s.store_objects(basic_quals_dict=basic_quals_dict)
print(f'"{child_str}" in basic_quals_dict: {child_str in basic_quals_dict}')

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\basic_quals_dict.pkl
"<li>Selenium</li>" in basic_quals_dict: False



----

In [318]:

# You've made no changes to the qualification dictionary (regardless of parts-of-speech changes)
from IPython.display import clear_output

def do_cypher_tx(tx, file_name, verbose=False):
    cypher_str = """
        MATCH (fn:FileNames {file_name: $file_name})
        SET fn.is_verified = true
        RETURN fn;"""
    if verbose:
        clear_output(wait=True)
        print(cypher_str.replace('$file_name', f'"{file_name}"'))
    parameter_dict = {'file_name': file_name}
    results_list = tx.run(query=cypher_str, parameters=parameter_dict)
    values_list = []
    for record in results_list:
        values_list.append(dict(record.items()))

    return values_list
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(do_cypher_tx, file_name=file_name, verbose=True)
row_objs_list


        MATCH (fn:FileNames {file_name: "HPC_User_Environment_Architect_Remote_Friendly_San_Francisco_Bay_Area_CA_Indeed_com.html"})
        SET fn.is_verified = true
        RETURN fn;


[{'fn': <Node id=860222 labels=frozenset({'FileNames'}) properties={'file_name': 'HPC_User_Environment_Architect_Remote_Friendly_San_Francisco_Bay_Area_CA_Indeed_com.html', 'percent_fit': 0.25, 'posting_url': 'https://www.indeed.com/pagead/clk/dl?mo=r&ad=-6NYlbfkN0CEBTCgjErpf876eNxykYOh7-CaikwoNOL0uEigY9r56FriOYldzu27sJiE-XsdX8X5nFxJgHgShIZchXzLQVAdcmACLaH_wjn6Lr0Bg9XOKE629BGDK70UrZ6aJKXrwjLdTNtOCnXUHuZdnZCWNpae5bce4Z9TSzyl4Q46askbPuIyerrXA7vKHTAuab--AjIC_P5hHd90ZthdKxScGJJgFRCiaZLIgBg6MGwDPuaoCCr45Ub0dI89KDb4KbxjHZUXnphMoMIP11fnQ5y5ck6uh9EegCtkQrpmFPaSc5R_kkzztu4PDRhnm0T5ypM0NHXC5_hDS8Fw_ZYCkhDaBY-z6yc9xQhArsPx2CXySNmpV01QzljZvwOAcxnm36x_Xrk_lln3Z7ONnXOruH778MaOya2IouN4nP-XkXqU53Cu18DwNsYSEOnbu87eeEM6CiPReAfn-ds9t6Uc06adWBjRLf95utvEZTxSGy4md1j8IbDexZXYe06O5VbIhr0jftBnMTUeJ_c-DKQm19MFF1VPjru20IKp2Uy4EUh7rKj_eL_woV4IZCfYEPa7f2MV-To97H3RF6Nj7_2MqT9Xhg5hIBoUsu5_mWunmzG5tEivlRbguhRCSq5Dj-rb2vrwT1c%3D&xkcb=SoCD-_M3VMZDXZ68JR0LbzkdCdPP&p=0&jsa=6737&rjs=1&tmtk=1gkc8j1ekjiph801&gdfvj=1&alid=62

In [287]:

# Mark the file name as needing retraining everywhere
import numpy as np
from IPython.display import clear_output

# file_name = '6b754ceaefd0a1f3_Lead_Data_Engineer_Brooklyn_NY_11201_Indeed_com.html'
mask_series = lru.hunting_df.percent_fit.isin([file_name])
lru.hunting_df.loc[mask_series, 'percent_fit'] = np.nan
s.store_objects(hunting_df=lru.hunting_df)
def do_cypher_tx(tx, file_name, verbose=False):
    cypher_str = """
        MATCH (fn:FileNames {file_name: $file_name})
        SET fn.percent_fit = NULL, fn.is_verified = false
        RETURN fn;"""
    if verbose:
        clear_output(wait=True)
        print(cypher_str.replace('$file_name', f'"{file_name}"'))
    results_list = tx.run(query=cypher_str, parameters={'file_name': file_name})

    return [dict(record.items()) for record in results_list]
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(do_cypher_tx, file_name=file_name, verbose=False)
row_objs_list

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\hunting_df.pkl


[{'fn': <Node id=860227 labels=frozenset({'FileNames'}) properties={'file_name': 'eca2876d9f354127_Security_Software_Engineer_L5_Detection_Engineering_Remote_Indeed_com.html', 'posting_url': 'https://www.indeed.com/rc/clk/dl?jk=eca2876d9f354127&from=ja&qd=RnZhMybXSk4M3QtTVGXWoZj-R_bxcYib5xeGNtZ7GZasUz1TeBf8ptY6h5mvt8XBcXElUinZ1eC1LFleePp6L-whUQKNiQjcDa3cvYy9VEw&rd=mlcWWhoEB84Ug9aTgB647ngDVkFzbh8-xI2u2vwcbH0&tk=1gkc8j1ekjiph801&alid=6254377b33b425113af40aa2', 'is_verified': False}>}]

In [ ]:

# Mark the file name as closed
cypher_str = f'''
    MATCH (fn:FileNames {{file_name: "{file_name}"}})
    SET fn.is_closed = true
    RETURN fn;'''
print(cypher_str)
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
row_objs_list

In [ ]:

# Manually label the unscored qual
qualification_str = quals_list[13]
print(qualification_str)
basic_quals_dict = s.load_object('basic_quals_dict')
basic_quals_dict[qualification_str] = 0
s.store_objects(basic_quals_dict=basic_quals_dict)

In [162]:

# Remove file name from database
# file_name = '3c031ea6ad293e92_General_Service_Technician_Westborough_MA_01581_Indeed_com.html'
cu.delete_filename_node(file_name, verbose=True)


                MATCH (fn:FileNames {file_name: "H1U_lwB_n4cl3xxj6yBdOw_Senior_Data_Engineer_Dice_United_States_Remote.html"})
                DETACH DELETE fn;
../saves/html\H1U_lwB_n4cl3xxj6yBdOw_Senior_Data_Engineer_Dice_United_States_Remote.html removed
